In [1]:
import youtube_dl
import os
from pprint import pprint
from pathlib import Path
import pandas as pd
import subprocess
import sys
import openpyxl
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import shutil
import re
from tqdm import tqdm
from moviepy.editor import *
from converter import Converter
import glob
import cv2
import ffmpeg

In [2]:
#Reads balanced_train_segments_propertext.xlsx as a dataframe
df = pd.read_excel ('C:\\Users\\DSPLAB\\Desktop\\Alroy Data\\Youtube-dl proj\\balanced_train_segments_propertext.xlsx',header=3) #takes the 3rd row as the header column
#header number is the row number to be considered as header
selection=df['# YTID']
print (selection)
ytlink= "youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v="
print(ytlink)

0        --PJHxphWEs
1        --ZhevVpy1s
2        --aE2O5G5WE
3        --aO5cdqSAg
4        --aaILOrkII
            ...     
22155    zyqg4pYEioQ
22156    zz0ddNfz0h0
22157    zz8TGV83nkE
22158    zzlK8KDqlr0
22159    zzya4dDVRLk
Name: # YTID, Length: 22160, dtype: object
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=


In [ ]:
#TO DOWNLOAD!! clips from YTID 
os.chdir(r'D:\Alroy youtube proj')
for i in selection[6431:]: #fill in [:]. Continue off from the last video you downloaded, the index of the next video to download onwards
    biglink=ytlink+i #decided to download all future files in .mp4 format to prevent slicing errors
    print(biglink, 'is downloading rn') #prints the file that is in the process of being downloaded
    os.system(biglink+" >> YTcommand_prompt.txt") #it adds on text to existing .txt file, important to see the last video downloaded

In [ ]:
#worms, a new list of YTIDs of the files that are successfully downloaded, not linked to CMD file
new_list=os.listdir(r'D:\Alroy youtube proj')
new_list.remove('Snippets')
new_list.remove('YTcommand_prompt.txt')
print(len(new_list))
worms=[]
for flies in new_list:
    maggot=flies.rsplit('.', 2)
    
    if maggot[-2].startswith('f') and len(maggot[-2])==4:
        pupa=maggot[-3][-11:]
    else:
        pupa=maggot[-2][-11:]
        
    worms.append(pupa)
print(len(worms))

In [ ]:
#reading the main balanced_segments excel file as dataframe
startend = pd.read_excel (r'C:\Users\DSPLAB\Desktop\Alroy Data\Youtube-dl proj\balanced_train_segments_propertext.xlsx',header=3) #takes the 3rd row as the header column
startend[['# YTID', ' start_seconds', ' end_seconds']]

In [ ]:
#df_reduced is the filtered 'startend' dataframe as it consist only of the videos that are sucessfully downloaded, to get start and end timings
df_reduced = startend[startend['# YTID'].str.contains('|'.join(worms))] #something wrong with this matching code
print(df_reduced[['# YTID', ' start_seconds', ' end_seconds']])
print(len(df_reduced)) #to see the number of rows in the reduced dataframe matches the length of the list in worms

In [ ]:
#Returns a SET of files between Snippets and Main download folder that has NOT been sliced, it worked!
#Using find function
longname=os.listdir(r"D:\Alroy youtube proj") #List of long names
longname.remove("Snippets")
longname.remove("YTcommand_prompt.txt")
hj=os.listdir(r"D:\Alroy youtube proj\Snippets")
shortname=[] #Compare list of short names
for i in hj:
    yu=i.rsplit('.',1)
    shortname.append(yu[-2])
print("no. of shortnames:", len(shortname))
#print(shortname) #no duplicates, i checked
    
actuallySliced=[]
for g_g in longname: #longname
    for h_h in shortname:  #shortname, keyword
        idx=g_g.find(h_h) #returns the index of where the keyword is found in the string
        if idx!= -1: #only can put != -1, if not each iteration it tried to macth will append to the list
            actuallySliced.append(g_g)
print("no. of actuallySliced:", len(actuallySliced)) #number of actually sliced should be = no. of shortnames
#print(actuallySliced)

#Comparing both lists: actuallySliced and longname. Finding the contrasting differences = unsliced videos, both lists must be IDENTICAL to one another, only finds the exact difference
longset=set(longname)
shortset=set(actuallySliced)
unslicedset=shortset.symmetric_difference(longset)
# print(unslicedset)
print("no. of unsliced file:", len(unslicedset)) #always check: take this number + number of snippets i have = total number of files in downloaded folder
#idk why but actuallySlicedfile should be = shortnames. I havent detective yet
#if file name has special characters (emojis) in the beginning, file cannot be read, fps==0, zero division error, need convert set into list.

In [ ]:
#test can also use this code to find those files that hasnt been sliced
#why cant the short names EXACTLY match with the long names. there are some shortnames names that are lef tout, have no partner
didntMatch=[] #finding the long names why some (shortnames) YTIDs cannot be .find() in the longnames
for it1 in longname: #longname
    count=0
    for it2 in shortname: #shortname
        idx=it1.find(it2)
        if idx== -1: #only can put != -1, if not each iteration it tried to macth will append to the list
            count+=1
        if count==len(shortname):
            print(count)
            didntMatch.append(it1)

## Functions

In [ ]:
#match each ID under # YTID column to each file name in downloaded file, returns a name
def findIdx(h): #depends on previously called df_reduced variable
    for a in df_reduced['# YTID']:
        idx=h.find(a) #inside bracket keyword, name returns an index
        if idx!= -1:
            name=h[idx:idx+11]
            return name

#returns a proper file extension format if the file ends with '.part', no need to use this anymore, now all .mp4
def ifPath(h): ##Getting the extension the video files for the slicing function, ext=ifPath(h)
    ext=h.split('.')
    if ext[-1]=='part':
        extention='.'+ ext[-2]
        return extention
    else:
        extention='.'+ ext[-1]
        return extention

#to convert mkv to mp4, dont use the subprocess function, use 'module'
class FFMConverter:
    def convert_mkv_mp4_subprocess(self, input_file, output_file):
        try: #ffmpeg -i test.mkv test.mp4
            command='ffmpeg -i' + input_file+' '+output_file
            subprocess.run(command)
            print("successully converted")
        except:
            print("Video failed to convert mkv to mp4")
            
    def convert_mkv_mp4_module(self, input_file, output_file):
        try:
            stream=ffmpeg.input(input_file)
            stream=ffmpeg.output(stream, output_file)
            ffmpeg.run(stream)
            print("successfully converted")
        except:
            print()
            
ffm=FFMConverter()
#ffm.convert_mkv_mp4_module('C:\\Users\\DSPLAB\\Desktop\\Shuffle vs hp hop--BxQ9nkrId4.mkv', "C:\\Users\\DSPLAB\\Desktop\\newcat"+".mp4") <--- very impt dont delete

#condition to convert MKV/webm file if detected, no need alr, all files are .mp4
def convertMKV(h,direct1,direct2): #direct1="C:\\Users\\DSPLAB\\Desktop\\tets2\\", locate the file itself    #direct2=""C:\\Users\\DSPLAB\\Desktop\\", end path   #h is the filename not filepath
    if h.endswith(".webm"):
        bare=h.removesuffix('.webm')  
        clipped=ffm.convert_mkv_mp4_module(direct1+h, direct2+bare+".mp4") #dont fking direct the converted file back to the same folder as the mkv file
        return clipped #returns the file

#creating another condition to remove '-' or '--' in front of filenames
def destroydash(eachfile):
    if eachfile[0]=='-':
        eachfile=eachfile.replace('-','')
    return eachfile

#Excel timings exceed video duration
def exceedTime(filename,end):
    try: 
        directory="D:\\Alroy youtube proj\\"   #"D:\\Alroy youtube proj\\"
        cap=cv2.VideoCapture(directory+filename)
        fps = cap.get(cv2.CAP_PROP_FPS)
        print("fps is:", fps)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        time=length/fps
        print("time is:", time)
        if end>time: #comparing end_clip time and max duration of video
            end=time
            print("new end time assigned, clip was in fact shorter than clip-endtime ")
        else: 
            print("clip-endtime is within clip duration")
    except:
        oof= open(filename+".txt","w+") #to keep track of the problematic filename
        oof.write("fps can't be read for some reason, time=length/fps, zero division error")
        oof.close()
    return end

#The Slicing function w/ OSERROR exception, moviepy error exception, create a txt file with its YTID in its place
def slicingCodeOS(filepath, start, end, endpath, newname):
    try:    
        video = VideoFileClip(filepath).subclip(t_start = start,t_end = end)
        video.write_videofile(endpath, logger=None)
        video.close() #need close
    except OSError:
        oof= open(newname+".txt","w+")
        oof.close()

## The Slicing Code

In [ ]:
#Edit this to clip Only the remaining files and not RECLIP existing files
for h in list(unslicedset):
    
    #creating single row dataframe for each YT file
    name=findIdx(h)
    minidf=df_reduced[df_reduced['# YTID'].isin([name])] 
    pprint(minidf[['# YTID', ' start_seconds', ' end_seconds']])
    
    #Renaming newfiles for the slicing function
    rawname=minidf.iloc[0]['# YTID']
    newname=destroydash(rawname)
    
    #Getting start and end timings to slice
    start=minidf.iloc[0][' start_seconds']
    end=minidf.iloc[0][' end_seconds']
    
    #Getting filepath of each file for the slicing function
    filepath="D:\\Alroy youtube proj\\"+h
    
    #Renaming the output clip
    endpath="D:\\Alroy youtube proj\\Snippets\\"+newname+".mp4"
   
    #Continue slicing even if clip-end timing exceed video duration
    end=exceedTime(h,end)
    
    #ffmpeg_extract_subclip(filepath,start,end, targetname=endpath) <- the OLD ultimate slicing function
    #The New Slicing Function
    slicingCodeOS(filepath, start, end, endpath, newname)
    print(h, 'has been clipped sucessfully')